Code complet, avec tous les archives, disponible sur git par le lien:
`https://github.com/Powerloleu/TP_Renforcement.git`

# MPAR - Rapport du rendu II
**01/04/24**

## Introduction

Ce notebook présente l'explication, la démonstration et les résultats des outils développés dans le cadre du cours de MPAR. Les résultats de la première partie du rendu, discutés en cours, ne seront pas abordés ici mais seront uniquement utilisés comme référence.

## Objectifs

1. Comprendre les concepts de **Model Checking** et de **SMC (Statistical Model Checking)** pour les **chaînes de Markov**.
2. Étendre ces concepts aux **MDP (Markov Decision Process)**.
3. Utiliser le **RL (Reinforcement Learning)** pour améliorer le model checking statistique.

## Contenu
1. **Changements dans le parser**
2. **Chapitre 2, Vérification Probabiliste**
3. **Chapitre 3, Modélisation Probabiliste et Apprentissage par Renforcement**

## 1. Changements dans le parser

Pour pouvoir utiliser des modèles .mdp avec des recompenses, il a fallut modifier notre parser. Pour ceci, les anciens modèles .mdp, sans récompenses, doivent continuer a fonctionnner sans altération.

De cette façon, nous avons inclu la ligne optionnelle `Rewards` avec des entrées dans le format `s:r` avec s un état et r une récompense entière. L'exemple `states_with_rewards` montre ceci.

Ceci a été fait en ajoutant l'entrée (defrewards) optionnele (?)  a la définition du programme, ayant `program: defstates defrewards? defactions transitions EOF;` et ayant la définition `defewrards : REWARDS ID ':' INT (',' ID ':' INT)* ';';`.

Après, nous avons ajouté l'attribut `rewards` à la classe `gramPrintListener`, défini a travers les fonctions `enterDefrewards` et `update_rewards`.

De cette façon, comme dans l'exemple vu en cours, il suffit d'importer la fonction `run` de l'archive `mdp.py` pour lire un fichier et creer un objet de la classe `gramPrintListener` qui contiendrat, en plus des attributs vus dans la prémière démostration, les attributs de récompenses.

In [5]:
from mdp import run as run_mdp # Fonction qui crée l'objet a être lu.

Exemple sans récompenses:

In [6]:
simu_mc = run_mdp(path = "prof_examples//simu-mc.mdp", return_printer=True, print_transactions=True) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
Initialy declared actions: ['a']
Transition from I with no action and targets ['T1', 'T2'] with weights [1, 1]
Transition from T1 with no action and targets ['T3', 'T4'] with weights [1, 1]
Transition from T2 with no action and targets ['T5', 'T6'] with weights [1, 1]
Transition from T3 with no action and targets ['S1', 'T1'] with weights [1, 1]
Transition from T4 with no action and targets ['S2', 'S3'] with weights [1, 1]
Transition from T5 with no action and targets ['S4', 'S5'] with weights [1, 1]
Transition from T6 with no action and targets ['S6', 'T2'] with weights [1, 1]
Transition from S1 with no action and targets ['S1'] with weights [1]
Transition from S2 with no action and targets ['S2'] with weights [1]
Transition from S3 with no action and

Exemple avec récompenses:

In [7]:
states_with_rewards = run_mdp(path = "mdp_examples//states_with_rewards.mdp", return_printer=True, print_transactions=True) # Exemple avec récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with action b and targets ['S1', 'S0'] with weights [2, 8]
Transition from S1 with action a and targets ['S2', 'S0', 'S1', 'S3'] with weights [1, 3, 6, 2]
Transition from S2 with action c and targets ['S0', 'S1', 'S3'] with weights [5, 5, 10]
Transition from S2 with action d and targets ['S0', 'S3'] with weights [5, 7]
Transition from S3 with action e and targets ['S1', 'S2'] with weights [2, 2]

 ------- transactions df -------
  Origin Action   S0   S1   S2    S3
0     S0     NA  NaN    5    5   NaN
1     S1      b    8    2  NaN   NaN
2     S1      a    3    6    1   2.0
3     S2      c    5    5  NaN  10.0
4     S2      d    5  NaN  NaN   7.0
5     S3      e  NaN    2

Comme montré dans les exemples précedents, à chaque état nous avons un reward attribué. 

Si la récompense n'a pas été définie, notre parser attribut une valeur de zero. 

Il est un dictionnaire et peut être appelé facilement par l'attribut `rewards` comme suit:

In [8]:
print('states_with_reards.mdp', states_with_rewards.rewards)
print('simu_mc.mdp', simu_mc.rewards)

states_with_reards.mdp {'S0': 1, 'S1': 10, 'S2': 15, 'S3': 0}
simu_mc.mdp {'I': 0, 'T1': 0, 'T2': 0, 'T3': 0, 'T4': 0, 'T5': 0, 'T6': 0, 'S1': 0, 'S2': 0, 'S3': 0, 'S4': 0, 'S5': 0, 'S6': 0}


## 2. Vérification Probabiliste (Chapitre 2)

### Exercice 12: 
Identification des ensembles $S_0$, $S_1$ et $S_?$ pour une propriété $P(\diamond s)$ (fatalment $s$, où $s$ est un état).

#### Solution:

Nous avons décidé d'utiliser notre dataframe de probabilitées pour raisoner. **Il marche avec des châines de Marcov et aussi des MDP**.

##### L'idée geral de notre algorithme est:

1. Au début, aucun état ne peut arriver: **$S_{0} = \{s \forall s \in G$ \}**. Nous ne deplaçons les états seulement depuis $S_0$ vers les autres ensembles, jamais le chemin inverse où de $S_?$ vers $S_1$ où vice versa.
2. L'état objectif satisfait toujours la propriété, alors **on déplace $source$ de $S_{0}$ pour $S_1$**
3. Maitenant, nous voulons **trouver tous les états $S_1$**. Pour ça, il faut faire **une boucle**.
    - Nous calculons la probabilité totale de chaque couple (état, action) d'arriver aux états de $S_1$. 
    - Pour chaque état, entre toutes les actions possibles qu'il peut avoir, si la probabilité totale minimum est de 1, alors toutes sont égals a 1, donc il est déplacé pour $S_1$.
    - Si $S_1$ a changé, nous recommençons le calcul. 
4. Maitenant, pour **trouver les états $S_?$**, nous refaisons **une boucle**.
    - Idem pour la probabilité d'arrivé, mais cette fois-ci en $S_1$ où $S_?$.
    - Ici, nous voulons l'ajouter a $S_?$ si au moins l'une de ces actions peut, peut-être, l'emmener a $S_?$ où $S_?$. Donc, il faut que la probabilité maximum d'un état soit plus grande que zéro.
    - Si $S_?$ a changé, nous recommençons le calcul. 


#### Idée de l'algo

L'idée de l'algorithme vient de la façon comme nous analisons un graphe pour trouver les ensembles. **D'abord, nous començons toujours par $S_1$ et puis après par $S_?$**. Aussi, a chaue fois nous regardons le graphe comme une seule chose au lieu de s'imaginer en marchant sur le graphe comme un walker.

In [9]:
def segment_suremaynever_states(printer, target_state):
    # Initialize sets for S_sure, S_may, and S_never
    s_sure = set()
    s_may = set()
    s_never = set(printer.declared_states)
    s_sure.add(target_state)    # origin is sure
    s_never.remove(target_state)
    
    stop = False
    # First cycle to add s_sures
    while not stop:
        stop = True
        probs = printer.transactions_prob.loc[:, ['Origin', 'Action']+list(s_sure)]
        probs['P_sure'] = probs.loc[:, list(s_sure)].sum(axis=1)
        for o in probs['Origin'].unique():
            if probs.loc[probs['Origin']==o]['P_sure'].min() == 1: # It's sure if all actions lead to 100% prob of arriving to sure. (So the "worst" action has prob 1)
                    if o in s_never:                               # If a new element is added, the df has changed, so we need to run again.
                        s_never.remove(o)
                        s_sure.add(o)
                        stop = False
    stop = False
    # First cycle to add s_sures
    while not stop:
        stop = True
        probs = printer.transactions_prob.loc[:, ['Origin', 'Action']+list(s_may)+list(s_sure)]
        probs['P_may'] = probs.loc[:, list(s_sure)+list(s_may)].sum(axis=1)
        for o in probs['Origin'].unique():
            if probs.loc[probs['Origin']==o]['P_may'].max() > 0: # It may arrive if at least one action have a probability of arriving to a may or sure (so the "best" action have a probability different than zero). 
                    if o in s_never:                             # If a new element is added, the df has changed, so we need to run again.
                        s_never.remove(o)
                        s_may.add(o)
                        stop = False
    
    return list(s_sure), list(s_may), list(s_never)

Testant l'algorithme pour le graphe suivant:

In [10]:
smaysurenever = run_mdp(path = "mdp_examples//smaysurenever.mdp", return_printer=True, print_transactions=False) 

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2', 'S3']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with action a and targets ['S1', 'S2'] with weights [1, 1]
Transition from S0 with action b and targets ['S0'] with weights [1]
Transition from S11 with no action and targets ['S2'] with weights [1, 1]
Transition from S2 with no action and targets ['S3'] with weights [1]
Transition from S1 with no action and targets ['S3'] with weights [1]
Transition from S5 with no action and targets ['S6'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - Undeclared state in transition: S11, declared automaticaly
( 1 ) - Undeclared state in transition: S5, declared automaticaly
( 2 ) - Undeclared state S6 targeted in transition from S5 with NA, declared automaticaly
( 3 ) - State S0 reward wasn't assigned, using zero as reward
( 4 ) - S

line 5:11 mismatched input ',' expecting {';', '+'}


![alt text](smaysurenever.png)

Image générée par l'archive `Plot_colors.py`.

Résultat:

In [11]:
S_sure, S_may, S_never = segment_suremaynever_states(smaysurenever, 'S3')
S_sure, sorted(S_may), sorted(S_never)

(['S2', 'S1', 'S3', 'S11'], ['S0'], ['S5', 'S6'])

### Exercice 13:

A partir d'un MDP $ M = (S, \text{Act}, P, \iota_{\text{init}}, AP, L) $ et une propriété $ P_{\text{max}} (\diamond s^*) $ avec $ s^* \in S $ un état donné. Proposer une définition pour la matrice $ A $ et le vecteur $ b $ du programme linéaire $ A \cdot x \geq b $. Calculer $x$.


#### Solution

Ici, nous voulons une solution gérale, qui marche pour des MDP mais aussi pour des MC. Pour ça, cest important de réetablire les vecteurs et matrices d'inéquations et équations a `None` quand nous n'avons pas d'équation où inéquation.

Pour résoudre le problème de programmation linéaire décrit, nous utilisons la bibliothèque `scipy.optimize` avec la fonction `linprog`, visant à minimiser la fonction objectif, qui est la somme des valeurs des variables d'état. L'implémentation de l'algorithme suit plusieurs étapes clés pour définir les contraintes et formuler le problème de manière à pouvoir être résolu efficacement.

- **Définition des Matrices et Vecteurs :**
  - `A_eq` et `b_eq` : Ces matrices et vecteurs sont utilisés pour représenter les contraintes d'égalité dans le problème de programmation linéaire. Dans le contexte de MDPs ou MCs, ces contraintes correspondent aux transitions d'état où il y a une certitude sur le résultat de l'action, c'est-à-dire lorsque de l'état source, il existe une unique transition possible. Ces contraintes garantissent que la somme des probabilités des transitions pour ces états source spécifiques est égale à un vecteur déterminé par la somme des probabilités vers les états `S_sure`.
  - `A_ub` et `b_ub` : Représentent les contraintes d'inégalité. Pour les états source avec plusieurs transitions possibles (dans `S_may`), ces contraintes sont définies pour refléter que la somme des probabilités de transition vers les états `S_may`, moins la probabilité de rester dans le même état, doit être inférieure ou égale aux probabilités de transition vers les états `S_sure`. Cela permet de gérer les incertitudes dans les transitions.

- **Logique et Principe de l'Algorithme :**
  L'algorithme parcourt tous les états source dans `S_may`. Pour chaque état, selon qu'il y ait une unique transition ou plusieurs, il applique une logique différente :
  - Si une unique transition est possible, la contrainte est ajoutée comme une égalité.
  - Si plusieurs transitions sont possibles, des contraintes d'inégalité sont ajoutées pour chaque transition possible.

Cela est suivi par la définition de la fonction objectif `c`, où nous cherchons à minimiser la somme des variables d'état (représentant les probabilités). 

Ensuite, les matrices et vecteurs de contraintes sont préparés pour s'assurer qu'ils sont dans le bon format pour `linprog`, et le problème de programmation linéaire est résolu en appelant `linprog` avec les paramètres définis.

Le résultat de `linprog` donne la distribution de probabilité optimale des états sous les contraintes données, minimisant ainsi la fonction objectif tout en satisfaisant les contraintes de transition entre les états.

Cette approche fournit une méthode systématique et efficace pour résoudre des problèmes complexes de décision stochastique en utilisant la programmation linéaire.


In [12]:
import numpy as np
from scipy.optimize import linprog

def solve_system(printer, S_may, S_sure):
    '''
    Résout un système de programmation linéaire défini par des contraintes sur les probabilités de transition entre les états d'un modèle de décision de Markov (MDP) ou d'une chaîne de Markov (MC).

    Parameters:
        printer (Printer): Un objet contenant les probabilités de transactions entre les états sous forme de DataFrame pandas.
        S_may (list): Liste des états où il y a potentiellement plusieurs transitions possibles.
        S_sure (list): Liste des états où une transition unique est sûre.

    Returns:
        res: Un objet OptimizeResult contenant le résultat de la résolution du problème de programmation linéaire.
        Ce résultat inclut la distribution optimale des probabilités d'état, la valeur de la fonction objectif à l'optimisation, un booléen indiquant si l'optimisation a réussi,
        et d'autres informations pertinentes sur le processus d'optimisation.

    Cette fonction établit et résout un problème de programmation linéaire pour trouver la distribution optimale des probabilités d'état qui minimisent la somme des probabilités dans les états incertains (S_may),
    tout en respectant les contraintes définies par les transitions sûres (S_sure) et les transitions potentielles (S_may) dans les modèles donnés.

    Les contraintes d'égalité sont utilisées pour les états avec une transition unique et sûre, tandis que les contraintes d'inégalité sont appliquées aux états avec plusieurs transitions potentielles.
    La fonction objectif vise à minimiser la somme des probabilités d'état dans S_may.

    Exemple d'utilisation:
        printer = Printer()  # Supposons que Printer est une classe définie ailleurs avec l'attribut transactions_prob.
        S_may = ['état1', 'état2']
        S_sure = ['état3', 'état4']
        resultat = solve_system(printer, S_may, S_sure)
        print(resultat)
    '''
     
    df = printer.transactions_prob
    # Initialize lists for inequality and equality constraints
    A_ub, b_ub, A_eq, b_eq = [], [], [], []
    Ubfollower = []
    # Iterate over source states in S_may
    for source_state in S_may:
        # Check if there is only one transition from the source state
        if len(df.loc[df['Origin'] == source_state]) == 1:
            # If only one transition, add it as an equality constraint
            t = df.loc[df['Origin'] == source_state, S_may].copy()
            t.loc[:, t.columns == source_state] -= 1  # Identity matrix substracted (A-I), but we invert later
            A_eq.append(-t.values)                    # (A-I) becomes (I-A)
            b_eq.append(np.sum(df.loc[df['Origin'] == source_state, S_sure].values, axis=1))
        else:
            # If multiple transitions, add them as inequality constraints
            mask_state = df['Origin'] == source_state
            Ubfollower.append(source_state)
            ts = df.loc[mask_state, S_may].copy()
            ts.loc[:, ts.columns == source_state] -=1
            Ubfollower.append(df.loc[mask_state, 'Action'].tolist())
            for i in range(len(df.loc[mask_state])):
                A_ub.append(ts.values[i])
                b_ub.append(-np.sum(df.loc[df['Origin'] == source_state, S_sure].values, axis=1)[i])
    
    c = np.ones(len(S_may)) # Objectif: Minimizer la somme des x de chaque état
        
    A_ub, b_ub, A_eq, b_eq = [None if not v else v for v in [A_ub, b_ub, A_eq, b_eq]]
    A_eq, A_ub = [np.vstack(m) if m is not None else None for m in [A_eq, A_ub]]

    # Solve the linear programming problem
    print('--------- System constraints:')
    print(f"{Ubfollower=}")
    print(f"{A_ub=}")
    print(f"{b_ub=}")
    print(f"{A_eq=}")
    print(f"{b_eq=}")
    print(f"{c=}")

    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0,1)) # Bounds établie des contraintes pour les valeurs de probabilité

    return res

In [13]:
def abc_analysis(printer, state):
    S_sure, S_may, S_never = segment_suremaynever_states(printer, state)
    S_sure, sorted(S_may), sorted(S_never)

    print('--------- S segments:')
    print(f'{S_sure=}')
    print(f'{S_may=}')
    print(f'{S_never=}')

    res = solve_system(printer, S_may, S_sure)

    print('--------- Solution:')
    print(f'{res.x=}')
    print(f'{res.fun=}')
    print(f'{res.success=}')
    print(f'{res.slack=}')
    print(f'{res.con=}')

#### Testant pour une MC:

In [14]:
mc_c2p41 = run_mdp(path = "mdp_examples//exemple_cours_c2p41_mc.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['II', 'CI', 'AI', 'CC', 'CA', 'AC', 'AA', 'F', 'L']
Initialy declared actions: ['NA']
Transition from II with no action and targets ['CI', 'AI'] with weights [2, 1]
Transition from CI with no action and targets ['CC'] with weights [1]
Transition from CC with no action and targets ['F', 'L'] with weights [1, 1]
Transition from CA with no action and targets ['F'] with weights [1]
Transition from AC with no action and targets ['F'] with weights [1]
Transition from AA with no action and targets ['L'] with weights [1]
Transition from AI with no action and targets ['AA'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State II reward wasn't assigned, using zero as reward
( 1 ) - State CI reward wasn't assigned, using zero as reward
( 2 ) - State AI reward wasn't assigned, using zero as reward
( 3 ) - State CC 

![alt text](exemple_cours_c2p41_mc.png)

Image générée par l'archive `Plot_colors.py`.

In [15]:
abc_analysis(mc_c2p41, 'F')

--------- S segments:
S_sure=['AC', 'F', 'CA']
S_may=['CC', 'CI', 'II']
S_never=['AI', 'L', 'AA']
--------- System constraints:
Ubfollower=[]
A_ub=None
b_ub=None
A_eq=array([[ 1.        , -0.        , -0.        ],
       [-1.        ,  1.        , -0.        ],
       [-0.        , -0.66666667,  1.        ]])
b_eq=[array([0.5]), array([0.]), array([0.])]
c=array([1., 1., 1.])
--------- Solution:
res.x=array([0.5       , 0.5       , 0.33333333])
res.fun=1.3333333333333333
res.success=True
res.slack=array([], dtype=float64)
res.con=array([0., 0., 0.])


#### Interprétation du résultat - MC:

Ici, dans une châine de Marcov, ce qui nous intérèsse est surtout le vecteur `res.x` qui rend les probabilitées des $S_?$. Nous avons:

$$P_{L} = 0.33, P_{AI} = 0.5, P_{AA} = 0.5$$

Le vecteur `res.con` étant zéro pour chaque inéquation montre que l'égalité a toujours été atteinte. Si ce n'était pas le cas, il n'y aurait pas de solution.

#### Testant pour un MDP simple

Pour le premier teste, nous avons choisi un MDP très simple pour que l'explication soit plus compréhensible.

In [16]:
simple_mdp = run_mdp(path = "mdp_examples//simple_mdp.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'W', 'L']
Initialy declared actions: ['a', 'b', 'c']
Transition from I with action a and targets ['W', 'L'] with weights [1, 1]
Transition from I with action b and targets ['W'] with weights [1]
Transition from I with action c and targets ['L'] with weights [1]
Transition from W with no action and targets ['W'] with weights [1]
Transition from L with no action and targets ['L'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State I reward wasn't assigned, using zero as reward
( 1 ) - State W reward wasn't assigned, using zero as reward
( 2 ) - State L reward wasn't assigned, using zero as reward


![alt text](simple_mdp.png)

Image générée par l'archive `Plot_colors.py`.

In [17]:
abc_analysis(simple_mdp, 'W')

--------- S segments:
S_sure=['W']
S_may=['I']
S_never=['L']
--------- System constraints:
Ubfollower=['I', ['a', 'b', 'c']]
A_ub=array([[-1.],
       [-1.],
       [-1.]])
b_ub=[-0.5, -1.0, -0.0]
A_eq=None
b_eq=None
c=array([1.])
--------- Solution:
res.x=array([1.])
res.fun=1.0
res.success=True
res.slack=array([0.5, 0. , 1. ])
res.con=array([], dtype=float64)


#### Interprétation du résultat - MDP simple:

Dans ce MDP, nous n'avos pas de contraintes d'égalitées. Ceci est du a cause du fait que l'unique état dans $S_{may}$ a des actions a choisir.

A nouveau, le vecteur x réprésente la probabilitée des états dans $S_{?}$. Dans les cas où il faut choisir un adversaire, cette probabilitée est la maximale (en choisissant le meilleur adversaire). Dans notre cas, elle est de $1$.

Le vecteur `res.slack` nous donne l'information sur quel adversaire a été choisi. Il répresente la différence dans chaque équation d'inégalité. **Pour les adversaires qui maximizent la probabilité, leur contrainte est active et donc la différence vaut zéro.** 

Pour interpréter qu'elle action est associé a la valeur du vecteur, nous avons le vecteur de strings `Ubfollower`. Avec les deux, nous povons voir que:

$$Slack_{I(a)} = 0.5, Slack_{I(b)} = 0, Slack_{I(c)} = 1,$$

Ce qui montre que l'adversaire optimale choisi $b$ dans l'état $I$. 

#### Teste pour un MDP plus complexe

In [18]:
mdp_c2p40 = run_mdp(path = "mdp_examples//exemple_cours_c2p40_mdp.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['II', 'CI', 'AI', 'CC', 'CA', 'AC', 'AA', 'F', 'L']
Initialy declared actions: ['NA', 'a', 'p', 'c']
Transition from II with action c and targets ['CI'] with weights [1]
Transition from II with action p and targets ['CI', 'AI'] with weights [2, 1]
Transition from II with action a and targets ['AI'] with weights [1]
Transition from CI with action a and targets ['CA'] with weights [1]
Transition from CI with action c and targets ['CC'] with weights [1]
Transition from CI with action p and targets ['CC', 'CA'] with weights [2, 1]
Transition from CC with no action and targets ['L', 'F'] with weights [1, 1]
Transition from CA with no action and targets ['F'] with weights [1]
Transition from F with no action and targets ['F'] with weights [1]
Transition from L with no action and targets ['L'] with weights [1]
Transition from AI with 

![alt text](exemple_cours_c2p40_mdp.png)

Image générée par l'archive `Plot_colors.py`.

In [19]:
abc_analysis(mdp_c2p40, 'F')

--------- S segments:
S_sure=['AC', 'F', 'CA']
S_may=['CC', 'CI', 'II', 'AI']
S_never=['L', 'AA']
--------- System constraints:
Ubfollower=['CI', ['a', 'c', 'p'], 'II', ['c', 'p', 'a'], 'AI', ['c', 'a', 'p']]
A_ub=array([[ 0.        , -1.        ,  0.        ,  0.        ],
       [ 1.        , -1.        ,  0.        ,  0.        ],
       [ 0.66666667, -1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        , -1.        ,  0.        ],
       [ 0.        ,  0.66666667, -1.        ,  0.33333333],
       [ 0.        ,  0.        , -1.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        , -1.        ]])
b_ub=[-1.0, -0.0, -0.3333333333333333, -0.0, -0.0, -0.0, -1.0, -0.0, -0.6666666666666666]
A_eq=array([[ 1., -0., -0., -0.]])
b_eq=[array([0.5])]
c=array([1., 1., 1., 1.])
--------- Solution:
res.x=array([0.5, 1. , 1. , 1. ])
res.fun=3.5

#### Interprétation du résultat - MDP complexe:

Dans ce MDP, nous n'avos:
- 9 contraintes d'inégalitées, 3 pour chaque état $\{II, CI, AI\}$.
- 1 contrainte d'égalité pour l'état $CC$.
A nouveau, le vecteur x réprésente la probabilitée des états dans $S_{?}$. Dans notre cas:

$$P_{II} = 1,  P_{CI}, P_{AI} = 1, P_{CC} = 1$$

Ce qui signifie que, avec un adversaire optimal, nous arrivons toujours en $F$.

Interprétant le vecteur `res.slack` et `Ubfollower`, nous avons: 
- Adversaire optimal de $II : \{c, p, a\} \to$ N'importe lequel de ces choix peux former un adversaire optimal, puisque les 3 contraintes sont actives (égales a zéro).
- Adversaire optimal de $CI: a \to$ L'état $a$ est l'unique qui optimize, donc notre adversaire doit toujours choisir $c$ dans l'état $CI$.
- Adversaire optimal $AI: c \to$ Idem que pour l'état $CI$ avec $c$.

Cette solution est intéressante parce que elle retourne toutes les résultats d'adversaires optimales possibles quand bien interprété. 

### Observation sur le format du résultat:
Nous avons préféré laisser un résultat plus "brute" pour nous obliger a bien comprendre la méthode de minimization derrière l'algorithme. Bien sûr, avec les vecteurs retournées, on aurait pû créer une solution directe facilement interprétable. Mais ça n'ajouté pas grand chose a la compréhension du problème.

### Exercice 14

Expliquer comment adapter les algorithmes de vérification de l'exercice 13 pour le calcul de la récompense attendue pour des modèles de récompense Markoviens (MC et MDP).

### Calcul de l'Espérance de Récompense pour les MC et MDP

Pour calculer l'espérance de récompense dans des modèles Markoviens, tels que les chaînes de Markov (MC) et les processus de décision de Markov (MDP), nous adoptons une approche basée sur les équations et inéquations qui intègrent les probabilités de transition entre états et les récompenses associées à ces transitions.

#### Chaînes de Markov (MC)

Pour une chaîne de Markov, l'espérance de récompense d'un état donné, notée $X_{\text{état}}$, est calculée via l'équation suivante :

$$X_{\text{état}} = \text{reward}_{\text{état}} + \sum_{\text{tous états}} (\text{probabilité de transition vers cet état} \times X_{\text{autres états}})$$

Ici, $X_{\text{état}}$ représente l'espérance de récompense pour l'état concerné, $\text{reward}_{\text{état}}$ est la récompense immédiate reçue en étant dans cet état, et la somme calcule l'espérance de récompense pondérée par les probabilités de transition vers tous les autres états.

#### Processus de Décision de Markov (MDP)

Dans le contexte des MDP, où les décisions (actions) influencent les transitions, l'espérance de récompense pour chaque action spécifique dans chaque état est définie par l'inéquation :

$$X_{\text{état}}^{\text{action}} \leq \text{reward}_{\text{état}} + \sum_{\text{tous états}} (\text{probabilité de transition vers cet état en utilisant l'action} \times X_{\text{autres états}})$$

$X_{\text{état}}^{\text{action}}$ désigne ici l'espérance de récompense pour un état donné lors de l'exécution d'une action spécifique. Cette formulation sous forme d'inéquation reflète le principe selon lequel, pour un MDP, l'espérance de récompense de choisir une action spécifique est contrainte par la somme de la récompense immédiate et de l'espérance de récompense des états suivants, cette dernière étant pondérée par les probabilités de transition utilisant l'action choisie.

L'emploi d'inéquations pour les MDP illustre le processus de choix et d'optimisation des actions : l'objectif est de déterminer la politique (c.-à-d., la sélection des actions dans chaque état) qui maximise l'espérance globale de récompense. Cela implique de maximiser la valeur de $X_{\text{état}}$ pour chaque état par le biais d'une sélection optimale d'actions.

Cette méthode offre un cadre pour modéliser et calculer l'espérance de récompense dans les MC et MDP, prenant en compte la structure décisionnelle et les transitions probabilistes entre états, et vise à optimiser les décisions en maximisant la récompense attendue.

L'implémentant sur python, nous avons:

In [20]:
import numpy as np
from scipy.optimize import linprog

def solve_system_with_rewards(printer, S_may, S_sure):     
    df = printer.transactions_prob
    # Initialize lists for inequality and equality constraints
    A_ub, b_ub, A_eq, b_eq = [], [], [], []
    Ubfollower = []
    # Iterate over source states in S_may
    for source_state in S_may:
        # Check if there is only one transition from the source state
        if len(df.loc[df['Origin'] == source_state]) == 1:
            # If only one transition, add it as an equality constraint
            t = df.loc[df['Origin'] == source_state, S_may].copy()
            t.loc[:, t.columns == source_state] -= 1  # Reward of the state
            A_eq.append(-t.values * np.array([printer.rewards[s] for s in S_may]))    # + expected reward * probability of other states 
            b_eq.append(printer.rewards[source_state])  # State contraint reward
        else:
            # If multiple transitions, add them as inequality constraints
            mask_state = df['Origin'] == source_state
            Ubfollower.append(source_state)
            ts = df.loc[mask_state, S_may].copy()
            ts.loc[:, ts.columns == source_state] -=1 # Reward of the state
            Ubfollower.append(df.loc[mask_state, 'Action'].tolist())
            for i in range(len(df.loc[mask_state])):
                A_ub.append(-ts.values[i]*np.array([printer.rewards[s] for s in S_may])) # sommatory of probs * rewards with -x added before
                b_ub.append(printer.rewards[source_state])
    
    c = -np.ones(len(S_may)) # Objectif: Minimizer l'éspérance de récompense dans chaque état 
        
    A_ub, b_ub, A_eq, b_eq = [None if not v else v for v in [A_ub, b_ub, A_eq, b_eq]]
    A_eq, A_ub = [np.vstack(m) if m is not None else None for m in [A_eq, A_ub]]

    # Solve the linear programming problem
    print('--------- System constraints:')
    print(f"{Ubfollower=}")
    print(f"{A_ub=}")
    print(f"{b_ub=}")
    print(f"{A_eq=}")
    print(f"{b_eq=}")
    print(f"{c=}")

    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0, None)) # Ici, il faut juste que les récompenses soyent positives.

    return res

# Observation:

Cette version est simplifié, elle calcule la récompense espérée d'arriver dans un état $S_{sure}$. Ça veut dire qu'elle ne comptabilize pas les récompenses entre le premier état $S_{sure}$ et l'état final souhaité.

In [21]:
def reward_analysis(printer, state):
    S_sure, S_may, S_never = segment_suremaynever_states(printer, state)
    S_sure, sorted(S_may), sorted(S_never)

    print('--------- S segments:')
    print(f'{S_sure=}')
    print(f'{S_may=}')
    print(f'{S_never=}')

    res = solve_system_with_rewards(printer, S_may, S_sure)

    print('--------- Solution:')
    print(f'{res.x=}')
    print(f'{res.fun=}')
    print(f'{res.success=}')
    print(f'{res.slack=}')
    print(f'{res.con=}')

### Exemple d'implémentation de l'algo avec des récompenses dans un MC

Ici, supposons que nous avons un board avec des cases de S0 à S3 et puis F. À chaque tour, on se déplace vers n'importe quelle case avec une probabilité unniforme (sauf la case où la pièce est, en autres mots elle se déplace toujours). Ici, la question qu'on veut répondre est combien de tours il faut en moyenne pour arriver en F.

In [22]:
mc_chemin_plus_court_arriver_en_f = run_mdp(path = "mdp_examples//mc_chemin_plus_court_arriver_en_f.mdp", return_printer=True, print_transactions=True)

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2', 'S3', 'F']
Initialy declared actions: ['b']
Transition from S0 with action b and targets ['S1', 'S2', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S1 with action b and targets ['S0', 'S2', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S2 with action b and targets ['S0', 'S1', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S3 with action b and targets ['S0', 'S1', 'S2', 'F'] with weights [1, 1, 1, 1]
Transition from F with no action and targets ['F'] with weights [1]

 ------- transactions df -------
  Origin Action   S0   S1   S2   S3  F
0     S0      b  NaN    1    1    1  1
1     S1      b    1  NaN    1    1  1
2     S2      b    1    1  NaN    1  1
3     S3      b    1    1    1  NaN  1
4      F     NA  NaN  NaN  NaN  NaN  1

 ------- transactions_prob df -------
  Origin Action    S0 

line 6:28 extraneous input '1' expecting {';', '+'}


![alt text](mc_chemin_plus_court_arriver_en_f.png)

Image générée par l'archive `Plot_colors.py`.

In [23]:
print(mc_chemin_plus_court_arriver_en_f.rewards)

{'S0': 1, 'S1': 1, 'S2': 1, 'S3': 1, 'F': 0}


In [24]:
reward_analysis(mc_chemin_plus_court_arriver_en_f, 'F')

--------- S segments:
S_sure=['F']
S_may=['S2', 'S0', 'S1', 'S3']
S_never=[]
--------- System constraints:
Ubfollower=[]
A_ub=None
b_ub=None
A_eq=array([[ 1.  , -0.25, -0.25, -0.25],
       [-0.25,  1.  , -0.25, -0.25],
       [-0.25, -0.25,  1.  , -0.25],
       [-0.25, -0.25, -0.25,  1.  ]])
b_eq=[1, 1, 1, 1]
c=array([-1., -1., -1., -1.])
--------- Solution:
res.x=array([4., 4., 4., 4.])
res.fun=-16.0
res.success=True
res.slack=array([], dtype=float64)
res.con=array([0., 0., 0., 0.])


#### Interprétation du résultat

Il est clairment correct. En première place parce que toutes les cases ont la même éspérance, ce qui est attendu en raison de la symétrie du problème. Pour vérifier si c'est bien 4, nous pouvons englober tous les états équivalents dans un état $S$ qui a probabilité $3/4$ de rester sur $S$ et $1/4$ de passer en $F$. De cette façon, l'ésperance de tours avant d'arriver en $F$ est de:

$$E(F) = 1\cdot \frac{1}{4} + 2 \cdot \frac{3}{4}\cdot \frac{1}{4} + ... + n \cdot \frac{3^{n-1}}{4^{n-1}} \cdot \frac{1}{4} $$
$$E(F) = \sum_{i=0}^{i=n} i \cdot \frac{3^{i-1}}{4^{i-1}} \cdot \frac{1}{4},n \to +\inf $$
$$E(F) - \frac{3}{4}E(F) = \sum_{i=0}^{i=1} 1 \cdot \frac{3^{i-1}}{4^{i-1}} \cdot \frac{1}{4},n \to +\inf $$
$$\frac{1}{4}E(F) = \frac{\frac{1}{4}}{1- \frac{3}{4}} $$
$$E(F) = 4$$

### Exemple d'implémentation de l'algo avec des récompenses dans un MDP

Ici, supposons que nous avons un board avec des cases de S0 à S3 et puis F. À chaque tour, nous choisissons de faire un pas en avant (F est après S3, c'est la dernière case) où de se déplacer vers n'importe quelle case avec une probabilité unniforme (sauf la case où la pièce est, en autres mots elle se déplace toujours). Ici, la question qu'on veut répondre est quel adversaire choisir pour optimizer le nombre de tours.

In [25]:
chemin_plus_court_arriver_en_f = run_mdp(path = "mdp_examples//chemin_plus_court_arriver_en_f.mdp", return_printer=True, print_transactions=True)

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2', 'S3', 'F']
Initialy declared actions: ['a', 'b']
Transition from S0 with action a and targets ['S1'] with weights [1]
Transition from S0 with action b and targets ['S1', 'S2', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S1 with action a and targets ['S2'] with weights [1]
Transition from S1 with action b and targets ['S0', 'S2', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S2 with action a and targets ['S3'] with weights [1]
Transition from S2 with action b and targets ['S0', 'S1', 'S3', 'F'] with weights [1, 1, 1, 1]
Transition from S3 with action a and targets ['F'] with weights [1]
Transition from S3 with action b and targets ['S0', 'S1', 'S2', 'F'] with weights [1, 1, 1, 1]
Transition from F with no action and targets ['F'] with weights [1]

 ------- transactions df -------
  Origin Action

line 9:28 extraneous input '1' expecting {<EOF>, ';', '+', ID}


![alt text](chemin_plus_court_arriver_en_f.png)

Image générée par l'archive `Plot_colors.py`.

In [26]:
print(chemin_plus_court_arriver_en_f.rewards)

{'S0': 1, 'S1': 1, 'S2': 1, 'S3': 1, 'F': 0}


In [27]:
reward_analysis(chemin_plus_court_arriver_en_f, 'F')

--------- S segments:
S_sure=['F']
S_may=['S2', 'S0', 'S1', 'S3']
S_never=[]
--------- System constraints:
Ubfollower=['S2', ['a', 'b'], 'S0', ['a', 'b'], 'S1', ['a', 'b'], 'S3', ['a', 'b']]
A_ub=array([[ 1.  , -0.  , -0.  , -1.  ],
       [ 1.  , -0.25, -0.25, -0.25],
       [-0.  ,  1.  , -1.  , -0.  ],
       [-0.25,  1.  , -0.25, -0.25],
       [-1.  , -0.  ,  1.  , -0.  ],
       [-0.25, -0.25,  1.  , -0.25],
       [-0.  , -0.  , -0.  ,  1.  ],
       [-0.25, -0.25, -0.25,  1.  ]])
b_ub=[1, 1, 1, 1, 1, 1, 1, 1]
A_eq=None
b_eq=None
c=array([-1., -1., -1., -1.])
--------- Solution:
res.x=array([2.        , 2.33333333, 2.33333333, 1.        ])
res.fun=-7.666666666666668
res.success=True
res.slack=array([0.        , 0.41666667, 1.        , 0.        , 0.66666667,
       0.        , 0.        , 1.66666667])
res.con=array([], dtype=float64)


#### Interprétation du résultat
A nouveau, les vecteurs d'intêret sont `res.x` et `res.slack`. 

A travers `res.x`, nous voyons l'éspérance du nombre de tours pour arriver en $F$ à partir d'un état donnée avec un adversaire qui minimize . Pour l'état $S3$, l'adversaire optimal aura une éspérance 1 par exemple.

Quelle stratégie ce adversaire aura? Pour ça, il faut analyser `res.slack` pour savoir quelles contrainter sont actives. Dans notre cas, pour l'état $S3$, l'adversaire optimal est de choisir l'action $a$.

Interpretant le résultat pour les autres états, nous avons:

$$Adversaire \space optimal = \{E_{S_0}(b) = 2.\overline{3}, E_{S_1}(b) = 2.\overline{3}, E_{S_2}(a) = 2, E_{S_3}(a) = 1\}$$

### Exemple d'implémentation de l'algo avec des récompenses dans un cas où elle peut être mal interprété

Ici, nous prénons un graphe de simulation de lancer d'un dé a travers des lancés de pièces où toutes les récompenses sont égales à 1.  De cette façon, minimizer les récompenses est equivalent aussi a l'idée de trouver l'éspérance de lancers jusqu'a ce qu'on trouve un numéro. Mais, bien sûr, l'algorithme marche pour des récompenses différentes de 1.

Ici, l'erreur viendra du fait que comme on n'arrive pas toujours au sommet, les récompenses attendues sont $+ \inf$ en réalité, mais l'algorithme ne montre pas ça.

In [28]:
chemin_plus_court_lancer_de_pieces = run_mdp(path = "mdp_examples//chemin_plus_court_lancer_de_pieces.mdp", return_printer=True, print_transactions=True)

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6']
Initialy declared actions: ['NA']
Transition from I with no action and targets ['S0', 'S1'] with weights [1, 1]
Transition from S0 with no action and targets ['S2', 'S3'] with weights [1, 1]
Transition from S1 with no action and targets ['S4', 'S5'] with weights [1, 1]
Transition from S2 with no action and targets ['F1', 'S0'] with weights [1, 1]
Transition from S3 with no action and targets ['F2', 'F3'] with weights [1, 1]
Transition from S4 with no action and targets ['S1', 'F4'] with weights [1, 1]
Transition from S5 with no action and targets ['F5', 'F6'] with weights [1, 1]

 ------- transactions df -------
  Origin Action    I   S0   S1   S2   S3   S4   S5   F1   F2   F3   F4   F5  \
0      I     NA  NaN    1    1  NaN  NaN  NaN  NaN  NaN  NaN  N

![alt text](chemin_plus_court_lancer_de_pieces.png "MC qui simule un dé à 6 faces")

In [29]:
print(chemin_plus_court_lancer_de_pieces.rewards)

{'I': 1, 'S0': 1, 'S1': 1, 'S2': 1, 'S3': 1, 'S4': 1, 'S5': 1, 'F1': 1, 'F2': 1, 'F3': 1, 'F4': 1, 'F5': 1, 'F6': 1}


In [30]:
# Analyse pour avoir 4:
reward_analysis(chemin_plus_court_lancer_de_pieces, 'F4')

--------- S segments:
S_sure=['F4']
S_may=['S4', 'S1', 'I']
S_never=['F1', 'F5', 'S0', 'F6', 'F3', 'S3', 'F2', 'S5', 'S2']
--------- System constraints:
Ubfollower=[]
A_ub=None
b_ub=None
A_eq=array([[ 1. , -0.5, -0. ],
       [-0.5,  1. , -0. ],
       [-0. , -0.5,  1. ]])
b_eq=[1, 1, 1]
c=array([-1., -1., -1.])
--------- Solution:
res.x=array([2., 2., 2.])
res.fun=-6.0
res.success=True
res.slack=array([], dtype=float64)
res.con=array([0., 0., 0.])


##### Interprétation

L'interprétation est similaire a celle de l'algorithme précedent. 
Ici, nous voyons que le nombre de lancers atttendu pour avoir S4 est de 2. 

Cette valeur, a prémière vue, semble incorrecte. Bien sûr, puisque c'est impossible d'atteindre S4 en 2 lancers, il faut au minimum 3.

Pourquoi la fonction retourne ceci? Parce que nous n'avons pas considéré les états $S_{never}$ dans notre solution. L'éspérance associé a eux est de $+\inf$. Donc, si nous les incluent, la réponse triviale est de $+\inf$ pour tous les états $S_{may}$. 

L'intuition est que notre algorithme considère que leur éspérance est de zéro. D'une certaine façon, si nous arrivons dans $S_{never}$, ça ne sert a rien de continuer a lancer des pièces. Donc, nous nous arrrêtons.

### Exercice 15
Simulateur de lancers de pièces successifs pour simuler un dé à 6 faces. Cette fois-ci prénant l'exemple utilisée par le professeur lors de la prémière évaluation.

In [31]:
de_6_faces = run_mdp(path = "prof_examples//simu-mc.mdp", return_printer=True, print_transactions=True)

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
Initialy declared actions: ['a']
Transition from I with no action and targets ['T1', 'T2'] with weights [1, 1]
Transition from T1 with no action and targets ['T3', 'T4'] with weights [1, 1]
Transition from T2 with no action and targets ['T5', 'T6'] with weights [1, 1]
Transition from T3 with no action and targets ['S1', 'T1'] with weights [1, 1]
Transition from T4 with no action and targets ['S2', 'S3'] with weights [1, 1]
Transition from T5 with no action and targets ['S4', 'S5'] with weights [1, 1]
Transition from T6 with no action and targets ['S6', 'T2'] with weights [1, 1]
Transition from S1 with no action and targets ['S1'] with weights [1]
Transition from S2 with no action and targets ['S2'] with weights [1]
Transition from S3 with no action and

![alt text](img_de_6_faces.png "MC qui simule un dé à 6 faces")

La fonction ci-dessous correspond à un générateur d'adversaires qui propose par défaut un choix aléatoire d'actions. Cette fonction nous servira lorsque nous mettrons en œuvre l'algorithme
qui traite des processus de décision markoviens.

In [32]:
import random

def gerar_preferencias_acoes(df, estados, acoes, modo="random"):
    """
    Generates a dictionary mapping each state to a list of preferred actions.

    This function allows for the specification of action preferences for each state
    within a Markov Decision Process (MDP), either through direct user input or
    by generating a random order of actions.

    Parameters:
    - df (pandas.DataFrame): A DataFrame containing the transition probabilities for each state-action pair.
    - estados (list): A list of all states declared in the MDP.
    - acoes (list): A list of all actions declared in the MDP.
    - modo (str): The mode of preference generation, either 'input' for user-defined preferences or 'random' for automatically generated preferences.

    Returns:
    - dict: A dictionary where keys are states and values are lists of actions, ordered by preference.
    """
    preferencias = {}

    for estado in estados:
        acoes_possiveis = df[df['Origin'] == estado]['Action'].unique()
        
        # Filtrar ações possíveis que não sejam 'NA', caso existam
        acoes_validas = [acao for acao in acoes_possiveis if acao != "NA"]
        
        if modo == "input":
            # Se houver apenas uma ação válida ou a ação é 'NA', seleção automática
            if len(acoes_validas) <= 1:
                preferencias[estado] = acoes_validas if acoes_validas else ["NA"]
            else:
                print(f"\nCurrent State: {estado}")
                print("Possible Actions: " + ", ".join(acoes_validas))
                preferred = input(f"Type in the preferred action for {estado} \n").strip()
                while preferred not in acoes_validas:
                    preferred = input(f"he inputed action {preferred} isn't available for this state, choose one in {acoes_validas} \n").strip()
                preferencias[estado] = [preferred]

        elif modo == "random":
            random.shuffle(acoes_validas)
            preferencias[estado] = acoes_validas
            
    return preferencias

In [33]:
def MonteCarlo_random_walk(p, target_state, num_transitions = 20):
    """
    Simulates a random walk through an MDP based on action preferences and transition probabilities.

    This function simulates traversing through a Markov Decision Process, making decisions at each state
    based on predefined or user-specified action preferences. It illustrates the potential path an agent
    might take, considering the MDP's transition probabilities for each action.

    Parameters:
    - p: An object containing the MDP structure, including its states, actions, transition probabilities, and other relevant data.

    Returns:
    - None: This function does not return a value but prints the simulation results, including the traversed path, actions taken, probabilities of transitions, and the total path probability.
    """
        
    df = p.transactions_prob
    found = False

    preferencias = gerar_preferencias_acoes(df, p.declared_states,p.declared_actions, modo="random")

    estado_atual = p.first_state

    caminho = [estado_atual]  # Iniciar o registro do caminho com o estado inicial
    probabilidade_acumulada = 1

    for _ in range(num_transitions):
        df_estado_atual = df[df['Origin'] == estado_atual]
        if df_estado_atual.empty:
            print("Stopped at a end of graph state")
            return
        acao_selecionada = None
        probabilidade_escolhida = None

        if df_estado_atual.iloc[0]['Action'] == "NA":
            probabilidades = df_estado_atual.iloc[0, 2:].astype(float).values
            acao_selecionada = "NA"
        else:
            for acao_preferida in preferencias[estado_atual]:
                df_acao_preferida = df_estado_atual[df_estado_atual['Action'] == acao_preferida]
                if not df_acao_preferida.empty:
                    probabilidades = df_acao_preferida.iloc[0, 2:].astype(float).values
                    acao_selecionada = acao_preferida
                    break

        probabilidades = probabilidades / np.sum(probabilidades)
        estados_possiveis = df_estado_atual.columns[2:]
        proximo_estado = np.random.choice(estados_possiveis, p=probabilidades)
        probabilidade_escolhida = probabilidades[np.where(estados_possiveis == proximo_estado)[0][0]]
        
        probabilidade_acumulada *= probabilidade_escolhida
        estado_passado = estado_atual
        estado_atual = proximo_estado
        caminho.append(estado_atual)  # Atualizar o caminho

        if target_state in caminho:
            found = True
            break


    return found

In [34]:
import numpy as np

if MonteCarlo_random_walk(de_6_faces, target_state="S1"):
    print("The target state was reached.")
else:
    print("The target state was not reached.")

The target state was reached.


### Exercice 16
Algorithme de SMC quantitatif - simulant un dé à 6 faces

In [35]:
def MonteCarloSimulator(p, target_state, num_simulations = 1000, num_transitions = 20):
    """
    Simulates a random walk through an MDP based on action preferences and transition probabilities.

    This function simulates traversing through a Markov Decision Process, making decisions at each state
    based on predefined or user-specified action preferences. It illustrates the potential path an agent
    might take, considering the MDP's transition probabilities for each action.

    Parameters:
    - p: An object containing the MDP structure, including its states, actions, transition probabilities, and other relevant data.

    Returns:
    - None: This function does not return a value but prints the simulation results, including the traversed path, actions taken, probabilities of transitions, and the total path probability.
    """
    found = 0
    for i in range(num_simulations):
        if MonteCarlo_random_walk(p, target_state, num_transitions):
            found += 1
    prob_reacing = round(found/num_simulations,4)
    print(f"Target state {target_state} was found in {found} out of {num_simulations} simulations. Probability of reaching it: {prob_reacing}")

In [36]:
MonteCarloSimulator(de_6_faces, target_state="S1", num_simulations=100, num_transitions=20)

Target state S1 was found in 16 out of 100 simulations. Probability of reaching it: 0.16


### Exercice 17.1

Implémenter l’algorithme de SMC qualitatif utilisant SPRT (Sequential Probability Ratio Test [Wald, 1945]) dans le programme de l’exercice 15.

In [37]:
def SPRT(p, target_state, p0, p1, alpha, beta, num_transitions=20):
    """
    Sequential Probability Ratio Test (SPRT) to determine if the probability of reaching a target state
    exceeds a specific threshold, with predefined significance levels and power.
    
    Parameters:
    - p: MDP or MC structure.
    - target_state: The target state we want to test for.
    - p0: The probability of success under the null hypothesis H0.
    - p1: The probability of success under the alternative hypothesis H1.
    - alpha: The significance level (probability of rejecting H0 when H0 is true).
    - beta: The test power (1 - probability of accepting H0 when H1 is true).
    - num_transitions: Maximum number of transitions per simulation.
    
    Returns:
    - A string indicating the test outcome (accept H0, accept H1, or continue data collection).
    """
    log_lambda = 0  # Log likelihood ratio
    A = np.log((1 - beta) / alpha)  # Threshold to accept H1
    B = np.log(beta / (1 - alpha))  # Threshold to accept H0
    

    while True:
        result = MonteCarlo_random_walk(p, target_state, num_transitions)
        
        # Calculate the log likelihood ratio based on the result
        if result:
            log_likelihood_ratio = np.log(p1 / p0)
        else:
            log_likelihood_ratio = np.log((1 - p1) / (1 - p0))
        
        log_lambda += log_likelihood_ratio

        if log_lambda >= A:
            return "Accept H1: The probability of reaching the target state is significantly high."
        elif log_lambda <= B:
            return "Accept H0: The probability of reaching the target state is not significantly high."
        # Otherwise, continue collecting data


- ##### "Accepter H1 : La probabilité d'atteindre l'état cible est significativement élevée."

Signification : Le test a déterminé, avec le niveau de confiance spécifié par $1 - \alpha$, que la probabilité d'atteindre l'état cible est significativement plus élevée que la probabilité sous l'hypothèse nulle $p_0$. Cela signifie que, sur la base des simulations effectuées, il existe des preuves suffisantes pour rejeter l'hypothèse nulle en faveur de l'hypothèse alternative. En d'autres termes, il est très probable que la véritable probabilité d'atteindre l'état cible soit supérieure à $p_0$, jusqu'au seuil défini par $p_1$.

- #### "Accepter H0 : La probabilité d'atteindre l'état cible n'est pas significativement élevée."

Signification : Le test a conclu, avec le niveau de confiance spécifié par $\beta$, qu'il n'y a pas de preuves suffisantes pour affirmer que la probabilité d'atteindre l'état cible est supérieure à la probabilité sous l'hypothèse alternative $p_1$. Par conséquent, l'hypothèse nulle n'est pas rejetée. Cela ne signifie pas nécessairement que l'hypothèse nulle est vraie, mais que, sur la base des simulations réalisées, il n'a pas été possible de démontrer que la probabilité de succès est significativement élevée comme défini par $p_1$. On peut dire que, dans le niveau de signification défini, la probabilité d'atteindre l'état cible ressemble davantage à $p_0$ ou est inférieure.


In [38]:
SPRT(de_6_faces, target_state="S1", p0=0.1, p1=0.15, alpha=0.01, beta=0.01, num_transitions=20)

'Accept H1: The probability of reaching the target state is significantly high.'

### Exercice 17.2
On appelle $p_{ki}$ la probabilité d’obtenir le chiffre $i$ après $k$ lancers au maximum. Utiliser votre programme pour estimer $P(p_{ki} \geq 0.1)$, $P(p_{ki} \geq 0.14)$, $P(p_{ki} \geq 0.16)$. Qu’observez-vous ? Utiliser $\alpha = \beta = 0.01$.

In [39]:
import numpy as np
from tqdm import tqdm

def estima_probabilites(p, target_state, seuils, alpha, beta, num_transitions=20, num_simulations=1000):
    """
    Estime les probabilités P(p_{ki} >= seuil) pour chaque seuil spécifié.
    
    Paramètres :
    - p : La structure MDP ou MC.
    - target_state : L'état cible à tester.
    - seuils : Une liste des seuils (p0) à tester.
    - alpha : Le niveau de signification pour rejeter H0.
    - beta : La puissance du test (1 - probabilité d'accepter H0 quand H1 est vrai).
    - num_transitions : Nombre maximum de transitions par simulation.
    - num_simulations : Nombre de simulations à exécuter pour chaque seuil.
    
    Retourne :
    - Un dictionnaire avec les seuils comme clés et les probabilités estimées comme valeurs.
    """
    resultats = {seuil: 0 for seuil in seuils}
    p1 = max(seuils) + 0.05  # Définir p1 comme une valeur un peu plus élevée que le seuil le plus élevé

    for seuil in seuils:
        accepte_h1 = 0
        
        for _ in tqdm(range(num_simulations), desc=f"Seuil {seuil}"):
            decision = SPRT(p, target_state, seuil, p1, alpha, beta, num_transitions)
            if decision == "Accept H1: The probability of reaching the target state is significantly high.":
                accepte_h1 += 1
        
        probabilite_estimee = accepte_h1 / num_simulations
        resultats[seuil] = probabilite_estimee
        
    return resultats


In [40]:
p = de_6_faces
target_state = "S1"
seuils = [0.1, 0.14, 0.16, 0.17]
alpha = 0.01
beta = 0.01
num_transitions = 10
num_simulations = 10

resultats = estima_probabilites(p, target_state, seuils, alpha, beta, num_transitions, num_simulations)
print(resultats)

Seuil 0.1:   0%|          | 0/10 [00:00<?, ?it/s]

Seuil 0.17: 100%|██████████| 10/10 [02:36<00:00, 15.67s/it]

{0.1: 0.8, 0.14: 0.2, 0.16: 0.0, 0.17: 0.0}


### Exercice 17.3
Tester sur le modèle du CRAPS.

In [47]:
craps = run_mdp(path = "mdp_examples//craps.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'P410', 'P59', 'P68', 'P', 'G']
Initialy declared actions: ['none']
Transition from I with no action and targets ['P', 'P410', 'P59', 'P68', 'G'] with weights [2, 3, 4, 5, 4]
Transition from P410 with no action and targets ['P', 'P410', 'G'] with weights [2, 9, 1]
Transition from P59 with no action and targets ['P', 'P59', 'G'] with weights [3, 13, 2]
Transition from P68 with no action and targets ['P', 'P68', 'G'] with weights [6, 25, 5]
Transition from G with no action and targets ['G'] with weights [1]
Transition from P with no action and targets ['P'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State I reward wasn't assigned, using zero as reward
( 1 ) - State P410 reward wasn't assigned, using zero as reward
( 2 ) - State P59 reward wasn't assigned, using zero as reward
( 3 ) - State P68 re

line 1:9 mismatched input ':' expecting {';', ','}


![alt text](craps.png)

In [48]:
SPRT(craps, target_state="G", p0=0.1, p1=0.15, alpha=0.01, beta=0.01, num_transitions=20)

'Accept H1: The probability of reaching the target state is significantly high.'

In [49]:
p = craps
target_state = "G"
seuils = [0.1, 0.14, 0.16, 0.17]
alpha = 0.01
beta = 0.01
num_transitions = 10
num_simulations = 10

resultats = estima_probabilites(p, target_state, seuils, alpha, beta, num_transitions, num_simulations)
print(resultats)

Seuil 0.1:   0%|          | 0/10 [00:00<?, ?it/s]

Seuil 0.17: 100%|██████████| 10/10 [00:09<00:00,  1.07it/s]

{0.1: 1.0, 0.14: 1.0, 0.16: 1.0, 0.17: 1.0}


### Exercice 17.4
Que faire si le modèle est un MDP

Notre code est suffisamment robuste pour traiter un MDP, puisque l'utilisation de Monte Carlo RandomWalk présuppose déjà le choix d'un adversaire aléatoire.

Il nous suffit d'instancier un MDP et d'appeler la fonction en lui passant un argument.

## 3. Modélisation Probabiliste et Apprentissage par Renforcement (Chapitre 3)

Ici, nous voulons implémenteer un algorithme simple de **Q-learning**.

In [55]:
class QLearningAgent:
    def __init__(self, n_states, n_actions, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        self.n_states = n_states
        self.n_actions = n_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.Q = np.zeros((n_states, n_actions))
    
    def choose_action(self, state, possible_actions):
        if np.random.uniform(0, 1) < self.epsilon:
            # Explore: choose a random action
            action = np.random.choice(possible_actions)
        else:
            # Exploit: choose the action with the highest Q-value for the current state
            action = np.argmax(self.Q[state])
        return action
    
    def learn(self, state, action, reward, next_state):
        # Q-learning update
        best_next_action = np.argmax(self.Q[next_state])
        td_target = reward + self.discount_factor * self.Q[next_state, best_next_action]
        td_delta = td_target - self.Q[state, action]
        self.Q[state, action] += self.learning_rate * td_delta

### Pour tester (choisir un état initial et un mdp dans le format printer):

In [62]:
def actions_of(printer, state):
    return printer.transactions_prob.loc[printer.transactions_prob['Origin'] == state, 'Action'].unique()

def apply_Q_learning(printer):
    n_states = len(printer.declared_states)
    n_actions =  len(printer.declared_actions)

    # Initialize the Q-learning agent
    agent = QLearningAgent(n_states, n_actions)

    # Training the agent
    n_episodes = 1000
    for episode in range(n_episodes):
        state = 0  # Initial state
        done = False
        while not done:
            print(printer.declared_actions)
            print(state)
            action = agent.choose_action(state, actions_of(printer,state))
            print(action)
            df = printer.transactions_prob
            df1 = df.loc[(df['Action']==action) & (df['Origin']==state), [col for col in df.columns if col not in ['Action', 'Origin']]]
            df1v = df1.values
            df1c = df1.columns
            next_state = np.random.choice(len(df1c), p=df1v.ravel())
            reward = printer.rewards[state]
            agent.learn(state, action, reward, next_state)
            state = next_state
            if state == 2:
                done = True
                
    # Print the learned Q-values
    print("Learned Q-values:")
    print(agent.Q)
